In [1]:
import json
from authorization import SpotifyAuth

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

In [5]:
spotify_authorisation = SpotifyAuth(client_id=client_id, client_secret=client_secret, redirect_url=redirect_url)
spotify_authorisation.set_access_token_from_file('access_token.json')

# Get User Details

In [6]:
import spotify_functions as spotify

In [7]:
spotify.get_user_details(spotify_authorisation.get_access_token())

{'display_name': 'sdspot2034',
 'external_urls': {'spotify': 'https://open.spotify.com/user/sdspot2034'},
 'href': 'https://api.spotify.com/v1/users/sdspot2034',
 'id': 'sdspot2034',
 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b8280153af8b79b3144b24f1291',
   'height': 64,
   'width': 64},
  {'url': 'https://i.scdn.co/image/ab6775700000ee8580153af8b79b3144b24f1291',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:sdspot2034',
 'followers': {'href': None, 'total': 9},
 'country': 'IN',
 'product': 'premium',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'email': 'shreyandas2034@gmail.com'}

In [8]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = int(yesterday.timestamp()*1e3)

# Get All Songs Played in the last one day

In [9]:
token = spotify_authorisation.get_access_token()
history = spotify.get_listening_history(token, cdc_time)['items']

## EDA over recent history

### Number of songs played

In [10]:
len(history)

12

### Fetch details of a song

#### See available data for a song

In [11]:
history[0].keys()

dict_keys(['track', 'played_at', 'context'])

In [12]:
list(history[0]['track'].keys())

['album',
 'artists',
 'disc_number',
 'duration_ms',
 'explicit',
 'external_ids',
 'external_urls',
 'href',
 'id',
 'is_local',
 'name',
 'popularity',
 'preview_url',
 'track_number',
 'type',
 'uri']

In [13]:
history[0]['played_at']

'2024-08-04T06:59:20.791Z'

In [14]:
print(history[0]['context'])

{'type': 'playlist', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/2m5BrqANdf443IYDvaYIRG?sessionId=1722729600003&mode=enhanced'}, 'href': 'https://api.spotify.com/v1/playlists/2m5BrqANdf443IYDvaYIRG', 'uri': 'spotify:playlist:2m5BrqANdf443IYDvaYIRG'}


# Transform

## Dimensions

### DIM_ARTIST

In [15]:
import pandas as pd
from sqlalchemy import Table, MetaData, Column, Integer, String
import sql_functions

In [16]:
# stacked list-comprehension
artist_list = [artist for artists in [play['track']['artists'] for play in history] for artist in artists]

In [17]:
df_artists = pd.DataFrame(artist_list)
df_artists = df_artists.drop(columns=['href','uri','external_urls'])
df_artists = df_artists.rename(columns={'id':'spotify_id','name':'artist_name','type':'artist_type'})
df_artists.drop_duplicates(inplace=True)
df_artists

,spotify_id,artist_name,artist_type
0,23zg3TcAtWQy7J6upgbUnj,USHER,artist
1,0TnOYISbd1XYRBk9myaseg,Pitbull,artist
2,0ekbDNE2eOq8QiaXM34wer,Howie Day,artist
3,5ILrArfIV0tMURcHJN8Q07,Vanessa Carlton,artist
4,6zvul52xwTWzilBZl6BUbT,Pixies,artist
5,1KCSPY1glIKqW2TotWuXOR,P!nk,artist
6,6fAEFVqFolcEhXeReay5u6,Loquat,artist
7,6deZN1bslXzeGvOLaLMOIF,Nickelback,artist
8,3AQRLZ9PuTAozP28Skbq8V,The Script,artist
9,2C9n4tQgNLhHPhSCmdsQnk,David Archuleta,artist


#### Load (Type 1 SCD)

In [18]:
metadata = MetaData()

# Define the dimension table schema
dim_table = Table('DIM_ARTIST', metadata,
    Column('artist_wid', Integer, primary_key=True, autoincrement=True),
    Column('spotify_id', String(255), nullable=False, unique=True),
    Column('artist_name', String(255)),
    Column('artist_type', String(255))
)

In [30]:
sql_functions.save_to_sql(df_artists, dim_table, key_cols=['spotify_id'], wid=['artist_wid'], mode='upsert')

### DIM_SONG

In [31]:
history[0]['track'].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [32]:
history[0]['track']

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/23zg3TcAtWQy7J6upgbUnj'},
    'href': 'https://api.spotify.com/v1/artists/23zg3TcAtWQy7J6upgbUnj',
    'id': '23zg3TcAtWQy7J6upgbUnj',
    'name': 'USHER',
    'type': 'artist',
    'uri': 'spotify:artist:23zg3TcAtWQy7J6upgbUnj'}],
  'external_urls': {'spotify': 'https://open.spotify.com/album/6A1F3Fkq5dYeYYNkXflcTX'},
  'href': 'https://api.spotify.com/v1/albums/6A1F3Fkq5dYeYYNkXflcTX',
  'id': '6A1F3Fkq5dYeYYNkXflcTX',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b27386b0c9728ad3ed338eaeea79',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e0286b0c9728ad3ed338eaeea79',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d0000485186b0c9728ad3ed338eaeea79',
    'width': 64}],
  'name': 'Raymond v Raymond (Expanded Edition)',
  'release_date': '2010-03-30',
  'release_date_precis

In [38]:
track_list = [play['track'] for play in history]

In [68]:
df_songs = pd.DataFrame(track_list)
df_songs = df_songs.explode('artists')
df_songs['artist_id'] = df_songs['artists'].apply(lambda x: x['id'])
df_songs['isrc_id'] = df_songs['external_ids'].apply(lambda x: x['isrc'])
df_songs = df_songs.drop(columns=['href','uri','external_urls','artists','external_ids', 'artist_id', 'popularity'])
df_songs['album'] = None
df_songs = df_songs.drop_duplicates()
df_songs = df_songs.rename(columns={'id':'song_id','album':'album_wid'})
df_songs

,album_wid,disc_number,duration_ms,explicit,song_id,is_local,name,preview_url,track_number,type,isrc_id
0,None,1,220800,False,4356Typ82hUiFAynbLYbPn,False,DJ Got Us Fallin' In Love (feat. Pitbull),https://p.scdn.co/mp3-preview/1619c7342806281d...,16,track,USLF21000041
1,None,1,249120,False,5aDpULK8MbJmHl42kR5KNI,False,Collide,https://p.scdn.co/mp3-preview/2f1f947c47d145da...,3,track,USSM10310894
2,None,1,237493,False,4w1lzcaoZ1IC2K5TwjalRP,False,A Thousand Miles,https://p.scdn.co/mp3-preview/1dab77144140b38c...,3,track,USIR10210955
3,None,1,229226,False,0KzAbK6nItSqNh8q70tb0K,False,Where Is My Mind?,https://p.scdn.co/mp3-preview/8414c9d0e363c390...,14,track,GBAFL9700100
4,None,1,203266,True,05qq6mwIbG1ZsQhlMKyDwN,False,Raise Your Glass,https://p.scdn.co/mp3-preview/e1749a65b6c70673...,17,track,USLF21000090
5,None,1,281386,False,4vR4jN5oYprr8lkQSlNu3M,False,Swingset Chain,https://p.scdn.co/mp3-preview/0044b25591dee368...,4,track,USJS10410118
6,None,1,223840,False,0gmbgwZ8iqyMPmXefof8Yf,False,How You Remind Me,https://p.scdn.co/mp3-preview/94fd201045f5b682...,2,track,NLA320119533
7,None,1,259880,False,0cnouzAiEjdjXB5xVVQ8Vo,False,Science & Faith,https://p.scdn.co/mp3-preview/77a51e7a1ef65ddc...,4,track,GBARL1000956
8,None,1,213520,False,4wCD0qOFNRu1Q6O3N6ycm4,False,Crush,https://p.scdn.co/mp3-preview/98ec5e65324cc981...,1,track,GBCTA0800201
9,None,1,215666,False,2727cNzjNBfVtjkDHLpuzr,False,End Of All Time,https://p.scdn.co/mp3-preview/699d9c6c002857c4...,2,track,USWU30900192


#### Load (Type 1 SCD)

In [69]:
from sqlalchemy import BigInteger, Boolean, Text


metadata = MetaData()

dim_song = Table(
    'DIM_SONG', metadata,
    Column('song_wid', Integer, primary_key=True, autoincrement=True),
    Column('song_id', String(62), nullable=False, unique=True),
    Column('album_wid', Integer),
    Column('disc_number', Integer),
    Column('duration_ms', BigInteger, nullable=False),
    Column('explicit', Boolean),
    Column('is_local', Boolean),
    Column('name', Text, nullable=False),
    Column('preview_url', Text),
    Column('track_number', Integer),
    Column('type', String(50)),
    Column('isrc_id', String(100))
)

In [70]:
sql_functions.save_to_sql(df_songs, dim_song, key_cols = ['song_id'], wid = ['song_wid'], mode = 'upsert')